![](https://raw.githubusercontent.com/UGS-CS/CSC3009/master/mashead.svg)

# <center>**Lab 04: Neural Networks**</center>

## **1. Data**



In this lab, you will build a simple perceptron network with the use of gradient descent algorithm.

Import libraries and settings

In [ ]:
# CSC3109 ML Team 16